In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
from time import time
import ijson

In [67]:
final = {}

In [68]:
num = 0
r = {}

In [5]:
#Rarity/cost of units 1-5
rarity = {"TFT3_TwistedFate": 0, "TFT3_Poppy": 0, "TFT3_Zoe": 0, "TFT3_Ahri": 1, "TFT3_Syndra": 2, "TFT3_WuKong": 3, "TFT3_Viktor": 3, "TFT3_Janna": 4, "TFT3_Leona": 0, "TFT3_Ashe": 2, "TFT3_Vayne": 2, "TFT3_Karma": 2, "TFT3_Irelia": 3, "TFT3_Riven": 3, "TFT3_Lulu": 4, "TFT3_Nocturne": 0, "TFT3_Annie": 1, "TFT3_Zed": 1, "TFT3_Shaco": 2, "TFT3_Rumble": 2, "TFT3_Fizz": 3, "TFT3_Illaoi": 0, "TFT3_Yasuo": 1, "TFT3_Blitzcrank": 1, "TFT3_Ezreal": 2, "TFT3_Vi": 2, "TFT3_Jinx": 3, "TFT3_Gnar": 3, "TFT3_AurelionSol": 4, "TFT3_Urgot": 4, "TFT3_Fiora": 0, "TFT3_JarvanIV": 0, "TFT3_Xayah": 0, "TFT3_Shen": 1, "TFT3_Rakan": 1, "TFT3_Cassiopeia": 2, "TFT3_Soraka": 3, "TFT3_Caitlyn": 0, "TFT3_Nautilus": 1, "TFT3_Jayce": 2, "TFT3_Teemo": 3, "TFT3_Jhin": 3, "TFT3_Lucian": 1, "TFT3_Ekko": 4, "TFT3_Neeko": 2, "TFT3_Mordekaiser": 1, "TFT3_Gangplank": 4, "TFT3_Thresh": 4, "TFT3_Malphite": 0, "TFT3_Graves": 0, "TFT3_KogMaw": 1, "TFT3_XinZhao": 1, "TFT3_Ziggs": 0, "TFT3_Xerath": 4, "TFT3_Darius": 1, "TFT3_Bard": 2, "TFT3_MasterYi": 2}

In [2]:
#sum up all numbers in the dictionary
def sum_round(moves):
    m = 0
    for i in moves:
        m += moves[i]
        
    return m

In [3]:
#Reading the export file
note = pd.read_json("export_10.json", lines=True)

In [ ]:
### Calculating by round data

In [4]:
#Filling in the dictionary with number of rows in the file and stage numbers
final = {}
for k in note:
    final[k] = {}
    final[k]['1-1'] = {}
    final[k]['1-2'] = {}
    final[k]['1-3'] = {}
    final[k]['1-4'] = {}
for i in final:
    for j in range(2, 8):
        for k in range(1, 8):
            reound = str(j) + '-' + str(k)
            final[i][reound] = {}

In [7]:
#For each row in the final calculate gold spent per round
for k in final:
    golds = note[k][0]['player']['gold']['by_round']
    last_round = note[k][0]['player']['final']['last_round']
    gold_spent = {}
    t = 1
    current_gold = 0
    prev_round = ''
    prev_gold = ''
    #For each round in the gold by round
    for i in golds:
        ro = i['current_round']
        go = int(i['gold'])
        #Ignore round if round 1-0
        if (ro != '1-0'):
            #If first recorded round
            if (t == 1):
                prev_round = ro
                prev_gold = go
                t = 2
            #If last round
            elif (ro == last_round):
                gold_spent[ro] = prev_gold - go
                final[k][ro]['gold_spent'] = gold_spent[ro]
            #else if round changes
            else:
                if (ro != prev_round):
                    gold_spent[prev_round] = prev_gold - current_gold
                    final[k][prev_round]['gold_spent'] = gold_spent[prev_round]
                    prev_round = ro
                    prev_gold = go
            current_gold = go
        
        

In [33]:
#For each row in final calculate health and health lost per round
for k in final:
    player = note[k][0]['playerName']
    health = note[k][0]['match']['players_health']
    prev_round = 0
    #For each round in health 
    for i in health:
        #If the health is the players health
        if (i['player'] == player):
            ro = i['current_round']
            #If first round
            if (ro == '1-0'):
                prev_round = 100
                final[k]['1-1']['health'] = i['health']
                final[k]['1-1']['hp_lost'] = 0
            else:
                final[k][ro]['health'] = i['health']
                final[k][ro]['hp_lost'] = prev_round - i['health']
                prev_round = i['health']

In [11]:
#For each row in final calculate movement frequency by round
for k in final:
    rounds = note[k][0]['player']['round_outcomes']['by_round']
    cell = {}
    r = '1-1'
    #If 1-0 not in rounds
    if '1-0' not in rounds:
        #for each round in match
        for i in rounds:
            moves = 0
            board = rounds[i]['board']
            #If board exists
            if board != None:
                #Go through list and check if unit has moved 
                for j in board:
                    unit = j['character_id']
                    if (unit in cell):
                        if (cell[unit] != j['cell']):
                            cell[unit] = j['cell']
                            moves += 1
                    else:
                        cell[unit] = j['cell']
                        moves += 1

            final[k][i]['move_freq'] = moves

In [16]:
#For each row in final extract rerolls by round
for k in final:
    reroll = note[k][0]['player']['rerolls']['by_round']
    for i in reroll:
        if i == '1-0':
            final[k]['1-1']['rerolls'] = reroll[i]
        else:
            final[k][i]['rerolls'] = reroll[i]

In [31]:
#For each row in final extract Units, Units bought and Units sold by round
for k in final:
    rounds = note[k][0]['player']['round_outcomes']['by_round']
    prev_units = {}
    t = 1
    #For each row in match
    for i in rounds:
        units = {}
        bought = 0
        sold = 0
        board = rounds[i]['board']
        bench = rounds[i]['bench']
        #If board exists
        if board != None:
            #Go through list calculate number of units
            for j in board:
                level = 1
                unit = j['character_id']
                #Level 1 units = 1 unit, level 2 units = 6 units, level 3 units = 9 units
                if j['level'] != '1':
                    level = int(j['level']) * 3
                if t == 1:
                    if unit not in prev_units:
                        prev_units[unit] = level
                    else:
                        prev_units[unit] += level
                else:
                    if unit not in units:
                        units[unit] = level
                    else:
                        units[unit] += level
        #If bench exists
        if bench != None:
            #Go through list calculate number of units
            for j in bench:
                level = 1
                unit = j['character_id']
                #Level 1 units = 1 unit, level 2 units = 6 units, level 3 units = 9 units
                if j['level'] != '1':
                    level = int(j['level']) * 3
                if t == 1:
                    if unit not in prev_units:
                        prev_units[unit] = level
                    else:
                        prev_units[unit] += level
                else:
                    if unit not in units:
                        units[unit] = level
                    else:
                        units[unit] += level
        #If first round units are just the total number of units 0 units bought and sold
        if t == 1:
            t = 2
            final[k][i]['units'] = sum_round(prev_units)
        #else calculate if units have been bought or sold
        else:
            diff = sum_round(units) - sum_round(prev_units)
            prev_units = units
            final[k][i]['units'] = sum_round(prev_units)
            if diff >= 0:
                bought = diff
            else:
                sold = diff * (-1)

        
        final[k][i]['bought'] = bought
        final[k][i]['sold'] = sold

In [57]:
test = {}
num = 1
#for each row in final record the row if it exists else do appropriate data cleaning
for i in final:
    prev_units = 0
    prev_bought = 0
    prev_sold = 0
    prev_health = 0
    for j in final[i]:
        if final[i][j]:
            test[num] = {}
            test[num]['round'] = j
            if 'gold_spent' in final[i][j]:
                test[num]['gold_spent'] = final[i][j]['gold_spent']
            else:
                test[num]['gold_spent'] = 0
            
            if 'health' in final[i][j]:
                test[num]['health'] = final[i][j]['health']
                test[num]['hp_lost'] = final[i][j]['hp_lost']
                prev_health = final[i][j]['health']
            else:
                test[num]['health'] = prev_health
                test[num]['hp_lost'] = 0
                
            if 'rerolls' in final[i][j]:
                test[num]['rerolls'] = final[i][j]['rerolls']
            else:
                test[num]['rerolls'] = 0
                
            if 'units' in final[i][j]:
                test[num]['units'] = final[i][j]['units']
                test[num]['bought'] = final[i][j]['bought']
                test[num]['sold'] = final[i][j]['sold']
                test[num]['move_freq'] = final[i][j]['move_freq']
                prev_units = final[i][j]['units']
                prev_bought = final[i][j]['bought']
                prev_sold = final[i][j]['sold']
            else:
                test[num]['units'] = prev_units
                test[num]['bought'] = prev_bought
                test[num]['sold'] = prev_sold
                test[num]['move_freq'] = 0
            num += 1

In [58]:
test

{1: {'round': '1-1',
  'gold_spent': 0,
  'health': 100.0,
  'hp_lost': 0,
  'rerolls': 0,
  'units': 1,
  'bought': 0,
  'sold': 0,
  'move_freq': 1},
 2: {'round': '1-3',
  'gold_spent': 4,
  'health': 100.0,
  'hp_lost': 0.0,
  'rerolls': 1,
  'units': 7,
  'bought': 6,
  'sold': 0,
  'move_freq': 2},
 3: {'round': '1-4',
  'gold_spent': 2,
  'health': 100.0,
  'hp_lost': 0.0,
  'rerolls': 1,
  'units': 12,
  'bought': 5,
  'sold': 0,
  'move_freq': 2},
 4: {'round': '2-1',
  'gold_spent': 3,
  'health': 96.0,
  'hp_lost': 4.0,
  'rerolls': 1,
  'units': 12,
  'bought': 5,
  'sold': 0,
  'move_freq': 0},
 5: {'round': '2-2',
  'gold_spent': 4,
  'health': 92.0,
  'hp_lost': 4.0,
  'rerolls': 1,
  'units': 20,
  'bought': 8,
  'sold': 0,
  'move_freq': 3},
 6: {'round': '2-3',
  'gold_spent': -1,
  'health': 86.0,
  'hp_lost': 6.0,
  'rerolls': 1,
  'units': 22,
  'bought': 2,
  'sold': 0,
  'move_freq': 1},
 7: {'round': '2-4',
  'gold_spent': 1,
  'health': 86.0,
  'hp_lost': 0.0,


In [59]:
#write the dictionary into a file
with open('by_round.txt', 'w') as outfile:
    json.dump(test, outfile)

In [ ]:
### end

In [ ]:
### One part of data_3.json

In [73]:
#For each row in note calculate the rarity and level of a players final units
#This variable was not used
for i in note:
    traits = note[i][0]['player']['final']['traits']
    if (len(traits) != 0): 
        main = max(traits, key = traits.get)
    else:
        main = "None"

    comp = note[i][0]['player']['final']['comp']
    levelOne = 0
    levelTwo = 0
    levelThree = 0
    starOne = 0
    starTwo = 0
    starThree = 0
    starFour = 0
    starFive = 0 
    for j in comp:
        if (j['level'] == '1'):
            levelOne += 1
        elif (j['level'] == '2'):
            levelTwo += 1
        elif (j['level'] == '3'):
            levelThree += 1

        star = rarity[j['character_id']] + 1
        if (star == 1):
            starOne += 1
        elif (star == 2):
            starTwo += 1
        elif (star == 3):
            starThree += 1
        elif(star == 4):
            starFour += 1
        elif(star == 5):
            starFive += 1
    
    num += 1
    r[num] = {}
    r[num]['main_trait'] = main
    r[num]['level_one'] = levelOne
    r[num]['level_two'] = levelTwo
    r[num]['level_three'] = levelThree
    r[num]['rarity_one'] = starOne
    r[num]['rarity_two'] = starTwo
    r[num]['rarity_three'] = starThree
    r[num]['rarity_four'] = starFour
    r[num]['rarity_five'] = starFive

In [74]:
print(len(r))

102000


In [71]:
del note

In [75]:
with open('data_3.txt', 'w') as outfile:
    json.dump(r, outfile)

In [76]:
count = 0
with open('data_3.txt','r') as file:
    data = json.load(file)

print(len(data))

102000


In [103]:
#For each round in note calculate the total movement frequency
for i in note:
    name = note[i][0]['puuid']
    game = note[i][0]['mid']
    rank = note[i][0]['player']['final']['placement']['rank']
    rounds = note[i][0]['player']['round_outcomes']['by_round']
    count = 0
    d = {}
    for j in rounds:
        for k in rounds[j]['board']:
            character = k['character_id']
            if character in d:
                if d[character]['cell'] != k['cell']:
                    d[character]['cell'] = k['cell']
                    count += 1
            else:
                d[character] = {}
                d[character]['cell'] = k['cell']
    num += 1
    r[num] = {}
    r[num]['game'] = game
    r[num]['player'] = name
    r[num]['move_freq'] = count
    r[num]['rank'] = rank

In [ ]:
### end

In [ ]:
### One part of the initial values calculated

In [128]:
#For each row in note get final rank, final gold gained, final total rerolls, total units bought in game
for i in note:
    name = note[i][0]['puuid']
    game = note[i][0]['mid']
    count = 0
    units = note[i][0]['player']['store']['total']['units_bought']['unit_bought']
    total = 0
    #Calculate total units bought in game
    for j in units:
        total += units[j]

    #Record appropriate stats
    if (game in final):
        final[game][name] = {}
        final[game][name]['game'] = game
        final[game][name]['rank'] = note[i][0]['player']['final']['placement']['rank']
        final[game][name]['gold'] = note[i][0]['player']['final']['gold_gained']
        final[game][name]['rerolls'] = note[i][0]['player']['rerolls']['total_rerolls']
        final[game][name]['units'] = total
    else:
        final[game] = {}
        final[game][name] = {}
        final[game][name]['game'] = game
        final[game][name]['rank'] = note[i][0]['player']['final']['placement']['rank']
        final[game][name]['gold'] = note[i][0]['player']['final']['gold_gained']
        final[game][name]['rerolls'] = note[i][0]['player']['rerolls']['total_rerolls']
        final[game][name]['units'] = total

In [ ]:
max(traits, key = traits.get)

In [ ]:
### end

In [ ]:
### Below are just various tests which I used to plan how I would extract the data

In [16]:
traits = note[41][0]['player']['final']['traits']
print(note[1870][0]['player']['final'])

{'comp': [{'character_id': 'TFT3_Zoe', 'character_name': 'Zoe', 'level': '1', 'items': [9], 'cell': 'cell_4'}], 'traits': {}, 'comp_traits': [], 'bench': [], 'last_round': '1-3', 'gold_left': '2', 'gold_gained': 2, 'gold_spent': 0, 'level': 2, 'xp_progress': '0/2', 'time_survived': 173.091, 'placement': {'rank': '0'}, 'name': 'NyanSilence'}


In [45]:
#Testing
comp = note[0][0]['player']['final']['comp']
levelOne = 0
levelTwo = 0
levelThree = 0
starOne = 0
starTwo = 0
starThree = 0
starFour = 0
starFive = 0 
for i in comp:
    print(i)
    if (i['level'] == '1'):
        levelOne += 1
    elif (i['level'] == '2'):
        levelTwo += 1
    elif (i['level'] == '3'):
        levelThree += 1
        
    star = rarity[i['character_id']] + 1
    if (star == 1):
        starOne += 1
    elif (star == 2):
        starTwo += 1
    elif (star == 3):
        starThree += 1
    elif(star == 4):
        starFour += 1
    elif(star == 5):
        starFive += 1
        
print(levelOne, levelTwo, levelThree)
print(starOne, starTwo, starThree, starFour, starFive)

{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}
{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}
{'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3, 77], 'cell': 'cell_15'}
{'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}
{'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}
{'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}
{'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}
{'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_28'}
1 7 0
2 2 1 2 1


In [129]:
j = 0
for i in final:
    print(i, final[i])
    break

VN1_3655815269 {'7863217': {'game': 'VN1_3655815269', 'rank': '4', 'gold': 298, 'rerolls': 49, 'units': 49}}


In [130]:
with open('data_1.txt', 'w') as outfile:
    json.dump(final, outfile)

In [131]:
count = 0
with open('data_1.txt','r') as file:
    data = json.load(file)
    
    for i in data:
        count += len(data[i])

print(len(final))
print(count)

88060
102000


In [126]:
del note

In [46]:
rounds = note[0][0]['player']['round_outcomes']['by_round']
count = 0
d = {}
for i in rounds:
    for j in rounds[i]['board']:
        character = j['character_id']
        if character in d:
            if d[character]['cell'] != j['cell']:
                d[character]['cell'] = j['cell']
                count += 1
        else:
            d[character] = {}
            d[character]['cell'] = j['cell']

In [54]:
r

{1: {'game': 'VN1_3655815269',
  'player': '7863217',
  'move_freq': 25,
  'rank': '4'},
 2: {'game': 'VN1_3655805705',
  'player': '18250278',
  'move_freq': 31,
  'rank': '3'},
 3: {'game': 'VN1_3655815278',
  'player': '14856370',
  'move_freq': 50,
  'rank': '3'},
 4: {'game': 'VN1_3655815321',
  'player': '1812899',
  'move_freq': 37,
  'rank': '1'},
 5: {'game': 'VN1_3655806749',
  'player': '605012',
  'move_freq': 43,
  'rank': '7'},
 6: {'game': 'VN1_3655815949',
  'player': '59765213',
  'move_freq': 37,
  'rank': '4'},
 7: {'game': 'VN1_3655816008',
  'player': '59934182',
  'move_freq': 25,
  'rank': '8'},
 8: {'game': 'VN1_3655806425',
  'player': '32369978',
  'move_freq': 22,
  'rank': '6'},
 9: {'game': 'VN1_3655815841',
  'player': '59585258',
  'move_freq': 21,
  'rank': '2'},
 10: {'game': 'VN1_3655806215',
  'player': '51802656',
  'move_freq': 8,
  'rank': '2'},
 11: {'game': 'VN1_3655806343',
  'player': '8070725',
  'move_freq': 38,
  'rank': '2'},
 12: {'game': 

In [33]:
note[0][0]['player']['round_outcomes']['by_round']

{'1-1': {'board': [{'character_id': 'TFT3_Fiora',
    'character_name': 'Fiora',
    'level': '1',
    'items': [3],
    'cell': 'cell_25'}],
  'bench': [{'character_id': 'TFT3_Nocturne',
    'character_name': 'Nocturne',
    'level': '1',
    'items': [],
    'cell': 'slot_1'},
   {'character_id': 'TFT3_Xayah',
    'character_name': 'Xayah',
    'level': '1',
    'items': [],
    'cell': 'slot_2'}],
  'opponent': None,
  'type': 'PVE',
  'outcome': 1.0,
  'max_champion_level': 1},
 '1-3': {'board': [{'character_id': 'TFT3_Fiora',
    'character_name': 'Fiora',
    'level': '1',
    'items': [],
    'cell': 'cell_25'}],
  'bench': [{'character_id': 'TFT3_Nocturne',
    'character_name': 'Nocturne',
    'level': '1',
    'items': [],
    'cell': 'slot_1'},
   {'character_id': 'TFT3_Xayah',
    'character_name': 'Xayah',
    'level': '1',
    'items': [],
    'cell': 'slot_2'},
   {'character_id': 'TFT3_Xayah',
    'character_name': 'Xayah',
    'level': '1',
    'items': [],
    'cell':

In [62]:
for i in note.loc[0]:
    index = i['patch']
    if index in patch:
        patch[index] += 1
    else:
        patch[index] = 1
print(patch)

{'10.18': 12000}


In [37]:
note.loc[0]

player             {'round_outcomes': {'by_round': {'1-1': {'boar...
match              {'carousel': [{'name': 'TFT3_Poppy', 'item_1':...
coaching           {'overwolf': {'all': [{'type': 'Highlights', '...
highlights         {'overwolf': {'good': [{'type': 'kda', 'messag...
scores             {'kda': 'A+', 'gold': 'A+', 'comp': 'B-', 'uni...
recommendations                                    {'recommend': []}
patch                                                          10.18
queueId                                                         1100
profileIconId                                                   4492
playerName                                                   yeukien
gameId                                                   3.65582e+09
region                                                           vn1
mid                                                   VN1_3655815269
puuid                                                        7863217
id                                

In [67]:
count = 0
for i in note[0][0]['player']['final']['comp']:
    print(i)
    if i['level'] == '1':
        count += 1
    else:
        if i['level'] == '2':
            count += 3
        else:
            count += 9
print(count)

{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}
{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}
{'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3, 77], 'cell': 'cell_15'}
{'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}
{'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}
{'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}
{'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}
{'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_28'}
22


In [36]:
patch = {}
for i, j in note.iterrows():
    if j.patch in patch:
        patch[j.patch] += 1
    else:
        patch[j.patch] = 1
print(patch)

{10.18: 2000}


In [7]:
ranks = {}
for i, j in note.iterrows():
    if j.player['final']['placement']['rank'] in ranks:
        ranks[j.player['final']['placement']['rank']] += 1
    else:
        ranks[j.player['final']['placement']['rank']] = 1
print(ranks)

{'4': 254, '3': 271, '1': 293, '7': 226, '8': 211, '6': 242, '2': 258, '5': 215, '0': 30}


In [44]:
match = {}
for i, j in note.iterrows():
    if j.mid in match:
        match[j.mid] += 1
    else:
        match[j.mid] = 1

In [48]:
tft_set = {}
for i, j in note.iterrows():
    if j.tft_set_number in tft_set:
        tft_set[j.tft_set_number] += 1
    else: 
        tft_set[j.tft_set_number] = 1
        
print(tft_set)

{3: 2000}


In [59]:
player = {}
for i, j in note.iterrows():
    if j.puuid in tft_set:
        player[j.puuid] += 1
    else: 
        player[j.puuid] = 1
        
print(len(player))

1259


In [19]:
units = note[0][0]['player']['store']['total']['units_bought']['unit_bought']
total = 0
for i in units:
    total += units[i]
total


49

In [38]:
note.loc[0].scores

{'kda': 'B-', 'gold': 'A+', 'comp': 'B-', 'units_contested': 'N/A'}

In [8]:
note_1 = pd.read_json("export_10.json")

In [6]:
note_1.loc[0]['player']['rerolls']

{'by_round': {'1-3': 1,
  '1-4': 1,
  '2-1': 1,
  '2-2': 1,
  '2-3': 1,
  '2-4': 1,
  '2-6': 1,
  '2-7': 1,
  '3-1': 1,
  '3-2': 1,
  '3-3': 1,
  '3-4': 4,
  '3-6': 6,
  '3-7': 4,
  '4-1': 3,
  '4-2': 1,
  '4-3': 4,
  '4-4': 1,
  '4-6': 1,
  '4-7': 1,
  '5-1': 1,
  '5-2': 1,
  '5-3': 1,
  '5-4': 9,
  '5-6': 9,
  '5-7': 1,
  '6-1': 1},
 'total_rerolls': 59}

In [4]:
patch = {}
for i, j in note_1.iterrows():
    if j.patch in patch:
        patch[j.patch] += 1
    else:
        patch[j.patch] = 1
print(patch)

{10.18: 10000}
